# USDA Food Environment Atlas Quickstart

This notebook demonstrates how to use the `USDAFoodAtlasConnector` to access food environment, food access, and food insecurity data from the USDA Economic Research Service.

**Copyright (c) 2025 Sudiata Giddasira, Inc. d/b/a Quipu Research Labs, LLC d/b/a KR-Labs™**  
**SPDX-License-Identifier: Apache-2.0**

## Setup

First, install the package and set up your API key:

```bash
pip install krl-data-connectors
export USDA_API_KEY="your_key_here"
```

Get your USDA API key at: https://api.data.gov/signup/

In [ ]:
from krl_data_connectors.agricultural import USDAFoodAtlasConnector
import pandas as pd
import matplotlib.pyplot as plt

# Initialize connector (uses USDA_API_KEY environment variable)
connector = USDAFoodAtlasConnector()

## Available Data Categories

The Food Environment Atlas provides data across multiple categories:

In [ ]:
# List all available categories
categories = connector.list_categories()
for code, description in categories.items():
    print(f"{code:15} - {description}")

## Get Food Access Data

Retrieve food access indicators for all counties in California:

In [ ]:
# Get food access data for California
ca_access = connector.get_county_data(category="access", state_fips="06")
print(f"Retrieved data for {len(ca_access)} California counties")
ca_access.head()

## Analyze Food Access Indicators

Let's analyze the percentage of population with low access to stores:

In [ ]:
# Get specific indicators
access_indicators = connector.get_indicators(
    indicators=["PCT_LACCESS_POP15", "PCT_LACCESS_CHILD15", "PCT_LACCESS_SENIORS15"],
    state_fips="06"
)

# Summary statistics
print("\nFood Access Statistics for California:")
print(access_indicators[["PCT_LACCESS_POP15", "PCT_LACCESS_CHILD15", "PCT_LACCESS_SENIORS15"]].describe())

## Food Insecurity Analysis

Analyze food insecurity rates across counties:

In [ ]:
# Get food insecurity data
ca_insecurity = connector.get_county_data(category="insecurity", state_fips="06")

# Sort by food insecurity rate
if "FOODINSEC_15_17" in ca_insecurity.columns:
    top_insecure = ca_insecurity.nlargest(10, "FOODINSEC_15_17")[["county", "FOODINSEC_15_17"]]
    print("\nTop 10 Counties by Food Insecurity Rate:")
    print(top_insecure)

## Store Availability Analysis

Compare grocery store and convenience store availability:

In [ ]:
# Get store availability indicators
stores = connector.get_indicators(
    indicators=["GROCERY14", "SUPERC14", "CONVS14"],
    state_fips="06"
)

# Visualize
if not stores.empty:
    fig, ax = plt.subplots(1, 3, figsize=(15, 4))
    
    stores["GROCERY14"].hist(bins=20, ax=ax[0])
    ax[0].set_title("Grocery Stores per 1,000 pop")
    ax[0].set_xlabel("Stores per 1,000")
    
    stores["SUPERC14"].hist(bins=20, ax=ax[1])
    ax[1].set_title("Supercenters per 1,000 pop")
    ax[1].set_xlabel("Stores per 1,000")
    
    stores["CONVS14"].hist(bins=20, ax=ax[2])
    ax[2].set_title("Convenience Stores per 1,000 pop")
    ax[2].set_xlabel("Stores per 1,000")
    
    plt.tight_layout()
    plt.show()

## Food Assistance Programs

Analyze SNAP (food stamps) participation rates:

In [ ]:
# Get food assistance data
ca_assistance = connector.get_county_data(category="assistance", state_fips="06")

if "SNAP_PART_RATE16" in ca_assistance.columns:
    # Sort by SNAP participation
    high_snap = ca_assistance.nlargest(10, "SNAP_PART_RATE16")[["county", "SNAP_PART_RATE16"]]
    print("\nTop 10 Counties by SNAP Participation Rate:")
    print(high_snap)
    
    # Visualize
    plt.figure(figsize=(10, 6))
    ca_assistance["SNAP_PART_RATE16"].hist(bins=30)
    plt.title("Distribution of SNAP Participation Rates in California Counties")
    plt.xlabel("SNAP Participation Rate (%)")
    plt.ylabel("Number of Counties")
    plt.show()

## Local Food Systems

Analyze farmers markets and local food infrastructure:

In [ ]:
# Get local food data
ca_local = connector.get_county_data(category="local", state_fips="06")

if "FMRKT16" in ca_local.columns:
    # Counties with most farmers markets per capita
    top_fmrkt = ca_local.nlargest(10, "FMRKT16")[["county", "FMRKT16"]]
    print("\nTop 10 Counties by Farmers Markets per 1,000 pop:")
    print(top_fmrkt)

## Health Indicators

Examine the relationship between food environment and health outcomes:

In [ ]:
# Get health indicators
ca_health = connector.get_county_data(category="health", state_fips="06")

if "PCT_DIABETES_ADULTS13" in ca_health.columns and "PCT_OBESE_ADULTS13" in ca_health.columns:
    # Scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(ca_health["PCT_OBESE_ADULTS13"], ca_health["PCT_DIABETES_ADULTS13"])
    plt.xlabel("Adult Obesity Rate (%)")
    plt.ylabel("Adult Diabetes Rate (%)")
    plt.title("Relationship between Obesity and Diabetes in California Counties")
    plt.grid(True, alpha=0.3)
    plt.show()

## Restaurant Availability

Compare fast food vs. full-service restaurants:

In [ ]:
# Get restaurant data
restaurants = connector.get_indicators(
    indicators=["FFR14", "FSR14"],
    state_fips="06"
)

if not restaurants.empty:
    # Scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(restaurants["FFR14"], restaurants["FSR14"], alpha=0.6)
    plt.xlabel("Fast Food Restaurants per 1,000 pop")
    plt.ylabel("Full-Service Restaurants per 1,000 pop")
    plt.title("Fast Food vs Full-Service Restaurant Availability")
    plt.grid(True, alpha=0.3)
    plt.show()

## Specific County Analysis

Deep dive into a specific county (Los Angeles County):

In [ ]:
# Get all data for Los Angeles County (FIPS: 06037)
la_data = connector.get_county_data(county_fips="06037")

print("\nLos Angeles County Food Environment Profile:")
print("=" * 50)

# Display key indicators
key_cols = [
    "PCT_LACCESS_POP15",
    "FOODINSEC_15_17",
    "SNAP_PART_RATE16",
    "GROCERY14",
    "FFR14",
    "FMRKT16",
]

for col in key_cols:
    if col in la_data.columns:
        value = la_data[col].iloc[0]
        print(f"{col:25} : {value}")

## Multi-State Comparison

Compare food environment metrics across multiple states:

In [ ]:
# Compare California, Texas, and New York
states = {"06": "California", "48": "Texas", "36": "New York"}

comparison_data = []
for fips, name in states.items():
    data = connector.get_county_data(state_fips=fips, category="insecurity")
    if "FOODINSEC_15_17" in data.columns:
        avg_insecurity = data["FOODINSEC_15_17"].mean()
        comparison_data.append({"State": name, "Avg Food Insecurity": avg_insecurity})

if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    print("\nAverage Food Insecurity Rate by State:")
    print(comparison_df)
    
    # Bar chart
    plt.figure(figsize=(10, 6))
    plt.bar(comparison_df["State"], comparison_df["Avg Food Insecurity"])
    plt.ylabel("Average Food Insecurity Rate (%)")
    plt.title("Food Insecurity Comparison Across States")
    plt.show()

## Next Steps

- Explore other categories: `socioeconomic`, `prices`, `restaurants`
- Combine with demographic data from Census connector
- Create time series analyses for trend identification
- Build predictive models for food access and health outcomes
- Export data for GIS mapping and spatial analysis

For more information:
- [USDA Food Environment Atlas](https://www.ers.usda.gov/data-products/food-environment-atlas/)
- [KRL Data Connectors Documentation](https://krl-data-connectors.readthedocs.io)
- [API Key Setup Guide](../API_KEY_SETUP.md)